## AgentCore Evaluations - Strands Agent를 위한 온라인 평가

이 튜토리얼에서는 Strands agent에 적용되는 AgentCore Evaluations의 온라인 평가 사용 방법을 배웁니다.

이 실습을 실행하려면 먼저 [00-prereqs](../../00-prereqs) 폴더의 코드를 사용하여 Strands agent를 생성하고 [01-creating-custom-evaluators](../../01-creating-custom-evaluators)의 코드를 사용하여 커스텀 evaluator를 생성해야 합니다.

### 학습 내용
- AgentCore Starter toolkit을 사용하여 trace에 대한 온라인 평가를 실행하는 방법

### 튜토리얼 세부 정보

| Information         | Details                                                                       |
|:--------------------|:------------------------------------------------------------------------------|
| Tutorial type       | 온라인 evaluator(내장 및 커스텀)를 사용한 Strands agent 평가                      |
| Tutorial components | 내장 및 커스텀 evaluator를 사용한 자동 평가 설정                                   |
| Tutorial vertical   | Cross-vertical                                                                |
| Example complexity  | Easy                                                                          |
| SDK used            | Amazon Bedrock AgentCore Starter toolkit                                      |

### 온라인 평가

온라인 평가는 배포된 agent의 실시간 트래픽 품질 모니터링을 가능하게 합니다. 특정 선택된 상호작용을 분석하는 온디맨드 평가와 달리, 온라인 평가는 실시간 트래픽을 기반으로 프로덕션 환경에서 agent의 성능을 지속적으로 평가합니다.

온라인 평가는 세 가지 주요 구성 요소로 이루어집니다. 첫째, **세션 샘플링 및 필터링**을 통해 agent 상호작용을 평가하기 위한 특정 규칙을 구성할 수 있습니다. 전체 세션의 일부(예: 10%)를 평가하기 위해 백분율 기반 샘플링을 설정하거나 더 타겟팅된 평가를 위해 조건부 필터를 정의할 수 있습니다. 둘째, 새로운 커스텀 evaluator 생성, 기존 커스텀 evaluator 사용 또는 내장 evaluator 선택을 포함한 **여러 평가 방법** 중에서 선택할 수 있습니다. 마지막으로, **모니터링 및 분석** 기능을 통해 대시보드에서 집계된 점수를 확인하고, 시간 경과에 따른 품질 추세를 추적하며, 낮은 점수의 세션을 조사하고, 입력에서 출력까지의 완전한 상호작용 흐름을 분석할 수 있습니다.

온라인 평가를 사용하면 특정 데이터 소스(agent trace를 포함하는 CloudWatch 로그 그룹 또는 AgentCore Runtime 엔드포인트)를 자동으로 모니터링하도록 시스템을 구성합니다. 서비스는 샘플링 및 필터링 규칙에 따라 들어오는 trace를 지속적으로 처리하고, 선택한 evaluator를 실시간으로 적용하며, 분석을 위해 상세한 결과를 CloudWatch에 출력합니다. 이 평가 유형은 프로덕션 모니터링, 품질 저하 조기 발견, 사용자 상호작용 패턴 식별 및 대규모 일관된 agent 성능 유지에 특히 유용합니다.

온라인 평가 구성을 생성하고 활성화하면 서비스가 백그라운드에서 지속적으로 실행되어 발생하는 세션을 평가하고 agent의 품질 메트릭에 대한 지속적인 가시성을 제공합니다. 필요에 따라 평가 전략을 조정하기 위해 언제든지 구성을 일시 중지, 수정 또는 삭제할 수 있습니다.

### Agent에서 AgentCore Observability로 trace 생성

AgentCore Observability는 상세한 실행 데이터를 캡처하고 구조화하기 위한 기반으로 [OpenTelemetry (OTEL)](https://opentelemetry.io/) trace를 활용하여 호출 중 agent 동작에 대한 포괄적인 가시성을 제공합니다. AgentCore는 다양한 agent 프레임워크에서 여러 유형의 OTEL trace를 계측하기 위해 [AWS Distro for OpenTelemetry (ADOT)](https://aws-otel.github.io/)에 의존합니다.

agent가 AgentCore Runtime에서 호스팅되는 경우(이 튜토리얼의 agent처럼), AgentCore Observability 계측은 최소한의 구성으로 자동으로 이루어집니다. `requirements.txt`에 `aws-opentelemetry-distro`를 포함하기만 하면 AgentCore Runtime이 OTEL 구성을 자동으로 처리합니다. agent가 AgentCore Runtime에서 실행되지 않는 경우, AgentCore Observability에서 사용할 수 있도록 ADOT로 계측해야 합니다. 텔레메트리 데이터를 CloudWatch로 전달하도록 환경 변수를 구성하고 OpenTelemetry 계측으로 agent를 실행해야 합니다.

프로세스는 다음과 같습니다:

![session_traces](../../images/observability_traces.png)

세션 trace가 AgentCore Observability에서 사용 가능해지면 AgentCore Evaluations를 사용하여 agent의 동작을 평가할 수 있습니다. 온라인 평가의 경우 추가 작업이 필요하지 않습니다. 라이브 대시보드에서 agent의 성능을 모니터링하기만 하면 됩니다.

### 온라인 평가가 trace와 작동하는 방식

온라인 평가에서 agent가 호출되고 AgentCore Observability에서 trace를 생성합니다. 이러한 trace는 세션에 매핑되고 해당 로그는 Amazon CloudWatch Log 그룹에서 사용할 수 있게 됩니다. 온라인 평가를 통해 개발자는 특정 agent에 대한 온라인 평가 구성을 생성하고 샘플 비율과 이 구성에 적용할 evaluator를 정의합니다. 그러면 AgentCore Evaluations가 프로덕션에서 agent를 자동으로 평가하여 설정된 샘플링 비율에 따라 생성된 trace를 분석합니다. 개발자는 AgentCore Observability 대시보드를 사용하여 agent의 trace 및 평가 점수를 시각화하고 평가 결과에 따라 agent를 지속적으로 업데이트할 수 있습니다.


![session_traces](../../images/online_evaluations.png)

### 이전 튜토리얼에서 정보 검색

이 튜토리얼에서는 사전 요구 사항 튜토리얼에서 AgentCore Runtime에 배포된 Strands agent를 사용합니다. 사전 구축된 메트릭과 `01-creating-custom-metrics` 튜토리얼에서 생성한 `response_quality` 메트릭으로 평가합니다. agent 및 evaluator 정보를 검색해 보겠습니다.

In [ ]:
# 이전 노트북에서 저장된 변수 불러오기
%store -r launch_result_strands
%store -r evaluator_id
try:
    print("Agent Id:", launch_result_strands.agent_id)
    print("Agent ARN:", launch_result_strands.agent_arn)
except NameError as e:
    raise Exception("""Missing launch results from your Strands agent. Please run 00-prereqs before executing this lab""")

try:
    print("Evaluator id:", evaluator_id)
except NameError as e:
    raise Exception("""Missing custom evaluator id. Please run 01-creating-custom-evaluators before executing this lab""")

### AgentCore Evaluations 클라이언트 초기화

이제 AgentCore Starter toolkit에서 AgentCore Evaluations 클라이언트를 초기화해 보겠습니다.

In [ ]:
from bedrock_agentcore_starter_toolkit import Evaluation, Observability
import os
import json
from boto3.session import Session
from IPython.display import Markdown, display

In [ ]:
# AWS 세션 생성 및 리전 정보 가져오기
boto_session = Session()
region = boto_session.region_name
print(region)

In [ ]:
# AgentCore Evaluations 클라이언트 초기화
eval_client = Evaluation(region=region)

### 온라인 평가 구성 설정

이제 온라인 평가 구성을 설정해 보겠습니다. 이 경우 데모 목적으로만 agent를 사용하므로 생성된 모든 trace를 평가합니다. 실제 애플리케이션에서는 agent의 활용도에 따라 샘플 비율을 적절히 설정해야 합니다.

온디맨드 평가에서 살펴본 5가지 메트릭으로 평가 구성을 생성합니다:
* Builtin.GoalSuccessRate
* Builtin.Correctness
* Builtin.ToolParameterAccuracy
* Builtin.ToolSelectionAccuracy 그리고
* 커스텀 메트릭: response_Quality

In [ ]:
# 온라인 평가 구성 생성
# sampling_rate=100: 모든 trace를 평가 (프로덕션에서는 적절히 조정 필요)
# auto_create_execution_role=True: 실행 역할 자동 생성
response = eval_client.create_online_config(
    agent_id=launch_result_strands.agent_id,
    config_name="strands_agent_eval2",
    sampling_rate=100,
    evaluator_list=[
        "Builtin.GoalSuccessRate", "Builtin.Correctness", 
        "Builtin.ToolParameterAccuracy", "Builtin.ToolSelectionAccuracy",
        evaluator_id
    ],
    config_description="Strands agent online evaluation test",
    auto_create_execution_role=True
)

### 평가 구성 분석

온라인 평가 구성의 구성 ID를 확인해 보겠습니다:

In [ ]:
# 생성된 온라인 평가 구성 ID 출력
print("Online Evaluation Configuration Id:", response['onlineEvaluationConfigId'])

생성된 구성의 세부 정보를 확인하여 이미 활성화되었는지 확인할 수도 있습니다:

In [ ]:
# 온라인 평가 구성 상세 정보 조회 (활성화 상태 확인)
eval_client.get_online_config(config_id=response['onlineEvaluationConfigId'])

### Agent 호출하여 평가 트리거

이제 몇 가지 새로운 쿼리로 agent를 호출하여 온라인 평가를 트리거해 보겠습니다. 이번에는 엔드포인트가 사용 가능해지면 모든 인터페이스에서 호출할 수 있으므로 boto3로 agent를 호출합니다.

In [ ]:
import boto3
# AgentCore 클라이언트 생성
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

def invoke_agent_runtime(agent_arn, prompt):
    # Agent Runtime 호출
    boto3_response = agentcore_client.invoke_agent_runtime(
        agentRuntimeArn=agent_arn,
        qualifier="DEFAULT",
        payload=json.dumps({"prompt": prompt})
    )
    # 응답이 event stream 형식인 경우 처리
    if "text/event-stream" in boto3_response.get("contentType", ""):
        content = []
        for line in boto3_response["response"].iter_lines(chunk_size=1):
            if line:
                line = line.decode("utf-8")
                if line.startswith("data: "):
                    line = line[6:]
                    print(line)
                    content.append(line)
        display(Markdown("\n".join(content)))
    else:
        # 일반 응답 형식 처리
        try:
            events = []
            for event in boto3_response.get("response", []):
                events.append(event)
        except Exception as e:
            events = [f"Error reading EventStream: {e}"]
        display(Markdown(json.loads(events[0].decode("utf-8"))))
    return boto3_response


In [ ]:
# Agent 호출 - 수학 계산 테스트
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "How much is 7+9+10*2?"
)

In [ ]:
# Agent 호출 - 날씨 질문 (도구 선택 테스트)
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "Is it raining?"
)

In [ ]:
# Agent 호출 - 백분율 계산 테스트
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "how much is 20% of 300?"
)

In [ ]:
# Agent 호출 - 기능 설명 요청
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "What can you do?"
)

In [ ]:
# Agent 호출 - 일반 지식 질문
response = invoke_agent_runtime(
    launch_result_strands.agent_arn,
    "What is the capital of NY State?"
)

### 온라인 평가 시각화

agent와 충분한 상호작용을 생성하면 [AgentCore Observability 콘솔](https://console.aws.amazon.com/cloudwatch/home#gen-ai-observability/agent-core/agents)을 사용하여 온라인 평가 구성에 따라 agent가 어떻게 수행되고 있는지 시각화할 수 있습니다.

agent의 `DEFAULT` 엔드포인트로 이동하여 현재 평가를 확인하세요.

**중요**: 평가 결과가 대시보드에 나타나는 데 시간이 걸릴 수 있습니다. 평가 대시보드가 비어 있으면 몇 분 후에 다시 확인하세요.

사용 가능해지면 agent의 trace에서 직접 메트릭을 볼 수 있습니다:
![image.png](../../images/online_evaluations_dashboard.png)

### 축하합니다!

첫 번째 온라인 평가 구성을 생성했습니다! 이제 커스텀 메트릭을 생성하고 AgentCore Evaluations를 사용하여 온디맨드 및 온라인으로 agent를 평가할 수 있습니다!